In [1]:
from typing import TypeVar
import sqlalchemy as sa
from sqlalchemy import NullPool
import pandas as pd

SelfClickHouseConnection = TypeVar("SelfClickHouseConnection", bound="ClickHouseConnection")

class ClickHouseConnection:
    """Класс контекстного менеджера для работы с ClickHouse"""
    
    def __init__(self, host: str, username: str, password: str, port: str = "9000"):
        """
        Параметры:
            host: хост ClickHouse
            username: имя пользователя ClickHouse
            password: пароль пользователя ClickHouse
            port: порт ClickHouse
        """
        
        self.session = None
        self.__host = host
        self.__port = port
        self.__username = username
        self.__password = password
        
    def __get_url(self) -> str:
        """Метод формирует connection string для подключения к ClickHouse"""
        
        url = "clickhouse+native://{username}:{password}@{host}:{port}"
        url = url.format(
            host = self.__host,
            port = self.__port,
            username = self.__username,
            password = self.__password
        )
        return url
        
    def __enter__(self: SelfClickHouseConnection) -> SelfClickHouseConnection:
        engine = sa.create_engine(self.__get_url(), poolclass=NullPool)
        self.session = engine.connect()
        return self
    
    def __exit__(self, exception_type, exception_value, traceback):
        self.session.close()
        
    def read_sql(self, query: str) -> pd.DataFrame:
        """
        Запрос к БД через фреймворк pandas
        
        Параметры:
            query: SQL в формате строки
            
        Возвращает:
            датафрейм с результатами запроса
        """
        
        with self as connector:
            return pd.read_sql(query, con=connector.session)

In [3]:
ch_host = "10.120.1.11"
ch_port = 8123
ch_username = "amarbuliev"
ch_password = "KuUNgVlPQiSN6dRqk"

connection_manager = ClickHouseConnection(host=ch_host, username=ch_username, password=ch_password)
df = connection_manager.read_sql("SELECT * FROM sandbox.ongoing_projects")

In [5]:
df.head()

,id,status,statusDesc,nameRus,head,directionHead,type,typeDesc,typeId,statusId,...,payed,projectOfficeControl,studentsCount,searchString,clientLogo,detailed_team,projectIndustryLabel,leaders,initiatorId,thumbnail
0,11,control_passed,Рабочий,Smart медицина,Фимина Ксения Игоревна,Белов Александр Владимирович,soft,Неизвестно,2,2,...,True,True,5,Фимина Ксения Игоревна 19058 Smart медицина Па...,,"[{'fullName': 'Гончаров Никита Павлович', 'nam...",Здравоохранение,"[{'id': 67, 'fio': 'Фимина Ксения Игоревна', '...",67.0,nan
1,17,control_passed,Рабочий,Оснащение Медиацентра МИЭМ,Елисеенко Андрей Михайлович,Королев Денис Александрович,soft-hard,Неизвестно,3,2,...,True,True,4,Елисеенко Андрей Михайлович 19102 Оснащение Ме...,,"[{'fullName': 'Асташов Степан Дмитриевич', 'na...",ИКТ,"[{'id': 633, 'fio': 'Елисеенко Андрей Михайлов...",NaN,nan
2,20,control_passed,Рабочий,Система бронирования аудиторий,Рыбаков Петр Владимирович,Рыбаков Петр Владимирович,soft,Неизвестно,2,2,...,True,True,2,Рыбаков Петр Владимирович 19105 Система бронир...,,"[{'fullName': 'Мартинич Андрей Иванович', 'nam...",ИКТ,"[{'id': 55, 'fio': 'Рыбаков Петр Владимирович'...",NaN,nan
3,21,control_passed,Рабочий,Mekstack: приватное облако,Емельяненко Максим Владимирович,Рыбаков Петр Владимирович,soft,Неизвестно,2,2,...,True,True,15,Емельяненко Максим Владимирович 19106 Mekstack...,,"[{'fullName': 'Автайкин Артём Олегович', 'name...",ИКТ,"[{'id': 1734, 'fio': 'Емельяненко Максим Влади...",NaN,https://cabinet.miem.hse.ru/project/thumbnail/21
4,24,control_passed,Рабочий,Цифровое портфолио,Сластников Сергей Александрович,Сластников Сергей Александрович,soft,Неизвестно,2,2,...,True,True,1,Сластников Сергей Александрович 19109 Цифровое...,,"[{'fullName': 'Дырченкова Юлия Александровна',...",Неизвестно,"[{'id': 62, 'fio': 'Сластников Сергей Александ...",NaN,nan
